In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import itertools
import math
import random


import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
data = pd.read_csv("offer_acceptance_offers.csv")
def conv_time(string):
    datetime_object = datetime.strptime(string, '%Y-%m-%d %H:%M:%S')
    return datetime_object

data['CREATED_ON_HQ'] = data['CREATED_ON_HQ'].apply(conv_time)
data.head()

,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,IS_OFFER_APPROVED,AUTOMATICALLY_APPROVED,MANUALLY_APPROVED,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,TRANSPORT_MODE,VALID
0,16badfc6202cb3f8889e0f2779b19218af4cbb736e56ac...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 09:05:43,11610.0,pool,False,False,False,False,False,False,False,False,FTL,True
1,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:54:13,9546.0,pool,True,False,False,False,False,False,False,True,FTL,True
2,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:57:27,9159.0,pool,True,True,False,True,False,True,True,True,FTL,True
3,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:57:28,9288.0,pool,False,False,False,False,False,False,False,False,FTL,True
4,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:57:40,9159.0,pool,False,True,False,True,False,False,False,False,FTL,True


## Cleaning of DataFrames (working on only FTL types)

In [3]:
FTL_data = data[data['TRANSPORT_MODE'] == 'FTL']
FTL_data = FTL_data.drop(['IS_OFFER_APPROVED', 'AUTOMATICALLY_APPROVED', 'MANUALLY_APPROVED'], axis=1)
FTL_data.shape

(358418, 12)

In [4]:
orders = pd.read_csv("offer_acceptance_orders.csv")
orders['ORDER_DATETIME_PST'] = orders['ORDER_DATETIME_PST'].apply(conv_time)
orders['PICKUP_DEADLINE_PST'] = orders['PICKUP_DEADLINE_PST'].apply(conv_time)

orders['diff_ORDER_PICKUP_DEADLINE'] = (orders['PICKUP_DEADLINE_PST'] - orders['ORDER_DATETIME_PST'])
orders.head()

/Users/nimayazdani/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9,10,11,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,REFERENCE_NUMBER,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,DELIVERY_TIME_CONSTRAINT,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE,diff_ORDER_PICKUP_DEADLINE
0,"[\n ""291ba58f4d48916df29008a6e451994650458661...",2022-09-07 10:07:05,2022-09-12 11:00:00,SCHEDULE,386,080,1035.0,22.0,True,True,False,True,True,0.0,True,3864.3240,FTL,5 days 00:52:55
1,"[\n ""e97bdf5ad99011a878f18c10b0df1f663f6b8f4a...",2022-05-19 09:21:39,2022-05-24 13:00:00,NONE,670,630,414.0,45.0,True,True,False,False,True,0.0,True,1411.1181,FTL,5 days 03:38:21
2,"[\n ""3cad7c1b5baf14968a6ee7c9b3649cb56193a40c...",2022-07-06 09:45:30,2022-07-08 14:00:00,SCHEDULE,952,853,703.0,20.0,True,True,False,False,False,0.0,True,2385.0681,FTL,2 days 04:14:30
3,"[\n ""c407879a0fc7dfe8dd40b8e0a8952eea0c5c850b...",2022-09-12 05:38:14,2022-09-19 10:30:00,SCHEDULE,381,346,753.0,52.0,True,False,False,False,False,0.0,False,2846.1141,FTL,7 days 04:51:46
4,"[\n ""e4ffed52c673af27ab35fcf4bb45e77becbbd95e...",2022-06-21 10:52:48,2022-07-05 13:00:00,SCHEDULE,917,953,345.0,48.0,True,True,False,False,False,0.0,False,1695.6405,FTL,14 days 02:07:12


In [5]:
joined = pd.merge(FTL_data, orders, on='REFERENCE_NUMBER', how='inner')
joined.sort_values(by = ['REFERENCE_NUMBER', 'CREATED_ON_HQ'], ascending = [True, True], na_position = 'first')
joined = joined[joined['OFFER_TYPE'] == 'quote']
joined


,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,...,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE_y,diff_ORDER_PICKUP_DEADLINE
0,ba3be1d6cdc455d085fabeafa3b5f12ce439c09fb7f25f...,"[\n ""46d8930a62fddd24e3d682d7231b19a4e2f8b5d8...",2021-08-31 10:09:23,3483.0000,quote,False,False,True,True,False,...,True,False,False,True,True,0.0,False,3528.2016,FTL,1 days 23:49:39
1,d302498475ec90c0b6da20bb1067dbcd62a40e59784727...,"[\n ""3b74ebf4c440a5ff141cafdc9b3e20230fc18360...",2021-10-08 05:41:19,4902.0000,quote,False,False,False,False,False,...,True,False,False,True,True,0.0,False,1995.4623,FTL,12 days 07:10:12
2,cb7593a6adb4b6339f9b0d794acc8903819db9e7c57624...,"[\n ""3b74ebf4c440a5ff141cafdc9b3e20230fc18360...",2021-10-08 09:51:09,2064.0000,quote,False,False,False,False,False,...,True,False,False,True,True,0.0,False,1995.4623,FTL,12 days 07:10:12
3,8cd6cb7e1601eb5dcd0a0d3d8f62c2dd8ee483819f9130...,"[\n ""3b74ebf4c440a5ff141cafdc9b3e20230fc18360...",2021-10-08 07:48:15,1935.0000,quote,False,True,False,False,False,...,True,False,False,True,True,0.0,False,1995.4623,FTL,12 days 07:10:12
4,10bb64fa1d284779a38ce9cc96b75ac7aa48c6a4bec7b2...,"[\n ""3b74ebf4c440a5ff141cafdc9b3e20230fc18360...",2021-10-08 09:56:44,1806.0000,quote,False,False,True,True,False,...,True,False,False,True,True,0.0,False,1995.4623,FTL,12 days 07:10:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265333,83753b9e6d8d4fb3ff4f966f0b8100d5c4789e57510013...,"[\n ""097c03d664562f70866e4781cf0486e1d9fddef2...",2021-09-24 10:53:30,174.1500,quote,False,False,True,True,False,...,True,True,False,True,True,0.0,True,6394.5300,FTL,5 days 04:19:17
265334,89a446ef8bb8ac97ca27da3615a282d67f9a95a1977684...,"[\n ""f7abeb819896fd044d2780d1888a526404094ef2...",2022-06-28 11:20:50,967.5000,quote,False,False,True,True,False,...,True,True,False,False,False,0.0,False,1157.9040,FTL,1 days 02:39:50
265335,3105e40f90f4fd0466bc8fc83304a6bb54f33cfe1e6635...,"[\n ""c3529a78b20fe2ec20939aae3bbc2bd83f16be9c...",2020-10-22 05:45:01,1032.0000,quote,False,False,True,True,False,...,True,True,False,True,True,0.0,False,1317.1674,FTL,1 days 05:59:35
265336,2e74b104e1bef1d96006a76e74259808c57b3f8113737a...,"[\n ""1b1c7fcc7402195342079a6ae31cfd20aa78f928...",2022-10-06 12:43:58,7095.0000,quote,False,False,False,False,False,...,True,False,False,False,True,0.0,False,5455.0746,FTL,12 days 02:33:52


In [6]:
# fill driving distance with mean of in zip code results, most na were in zip code
same = orders[orders['ORIGIN_3DIGIT_ZIP'] ==  orders['DESTINATION_3DIGIT_ZIP']]
mean_same_distance = same['APPROXIMATE_DRIVING_ROUTE_MILEAGE'].dropna().mean()
orders['APPROXIMATE_DRIVING_ROUTE_MILEAGE'] = orders['APPROXIMATE_DRIVING_ROUTE_MILEAGE'].fillna(mean_same_distance)

# fill FD enabled with false
orders['FD_ENABLED'] = orders['FD_ENABLED'].fillna(False)

# drop everything else, cause all had consistance NA 7
# 18 in Transportmode
# change all boolean data back to boolean
containsNa = (orders.isnull().sum() > 1).to_dict()
orders = orders.dropna()
for key,value in containsNa.items():
    if value:
        if orders[key].unique().sum() == 1:
            orders[key] = orders[key].astype(bool)

In [7]:
offers = pd.read_csv("offer_acceptance_offers.csv",low_memory=False)
offers.head()

,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,IS_OFFER_APPROVED,AUTOMATICALLY_APPROVED,MANUALLY_APPROVED,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,TRANSPORT_MODE,VALID
0,16badfc6202cb3f8889e0f2779b19218af4cbb736e56ac...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 09:05:43,11610.0,pool,False,False,False,False,False,False,False,False,FTL,True
1,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:54:13,9546.0,pool,True,False,False,False,False,False,False,True,FTL,True
2,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:57:27,9159.0,pool,True,True,False,True,False,True,True,True,FTL,True
3,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:57:28,9288.0,pool,False,False,False,False,False,False,False,False,FTL,True
4,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:57:40,9159.0,pool,False,True,False,True,False,False,False,False,FTL,True


In [8]:
s = offers.groupby('REFERENCE_NUMBER').count()['CARRIER_ID']
nOffers_rec = s[s < 15]

In [9]:
ftl = orders[orders['TRANSPORT_MODE'] == 'FTL']
ftl.set_index('REFERENCE_NUMBER',inplace = True)
joinedDF = ftl.join(nOffers_rec,how = 'left')
joinedDF['NUMBER_OFFERS'] = joinedDF['CARRIER_ID'].fillna(0)
joinedDF.drop('CARRIER_ID',axis = 1,inplace = True)
joinedDF['SECONDS_BETWEEN_ORDER_AND_DEADLINE'] = joinedDF['diff_ORDER_PICKUP_DEADLINE'].dt.total_seconds()
joinedDF


,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,DELIVERY_TIME_CONSTRAINT,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE,diff_ORDER_PICKUP_DEADLINE,NUMBER_OFFERS,SECONDS_BETWEEN_ORDER_AND_DEADLINE
REFERENCE_NUMBER,,,,,,,,,,,,,,,,,,,
"[\n ""000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c""\n]",2022-08-18 12:45:34,2022-08-24 11:00:00,SCHEDULE,761,304,1025.0,52.0,True,True,False,False,False,0.0,True,2353.6050,FTL,5 days 22:14:26,7.0,512066.0
"[\n ""0000c7c54e3a484cc8b20ea284522a37077143fd6aa056cbb9a0db8021e4eb89""\n]",2022-06-08 09:56:04,2022-06-20 13:00:00,SCHEDULE,314,071,792.0,51.0,True,True,False,False,False,0.0,True,3657.3564,FTL,12 days 03:03:56,0.0,1047836.0
"[\n ""0002e80902489c5a5eed029971f32e2ade3513870d804c653e8fcb3523a431aa""\n]",2021-11-15 09:36:08,2021-11-16 12:00:00,NONE,300,958,2509.0,52.0,True,True,False,True,True,0.0,False,5243.3082,FTL,1 days 02:23:52,4.0,95032.0
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,False,False,0.0,True,3024.8049,FTL,8 days 00:15:44,3.0,692144.0
"[\n ""0004b681b1c4f14dc289a36cf0590ffce5b6aa2220f6b340d550c952ea4caf23""\n]",2020-08-17 15:29:20,2020-08-18 15:30:00,NONE,900,276,2547.0,24.0,True,True,False,True,True,0.0,False,9245.7654,FTL,1 days 00:00:40,1.0,86440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"[\n ""fffbec3e1ffcf0c505877a2b68e8fc773a413f9db25cc40fb66eff17b333354a""\n]",2021-12-09 15:57:39,2021-12-13 15:30:00,NONE,917,463,2024.0,52.0,True,False,False,False,True,0.0,False,9373.3464,FTL,3 days 23:32:21,1.0,343941.0
"[\n ""fffc15629ba0aa26f5252d1b619e01cf10306f97b440027ee7dc899a4f59b107""\n]",2021-03-29 15:39:19,2021-03-31 11:00:00,SCHEDULE,303,945,2471.0,47.0,True,True,False,True,False,0.0,True,6936.4461,FTL,1 days 19:20:41,4.0,156041.0
"[\n ""fffdb6cd77e2d5e8bfa5a355cf80a2c90a1a800f086add7b12f7d0ae1e2ecdca""\n]",2021-08-23 12:42:55,2021-08-26 13:00:00,SCHEDULE,305,770,850.0,48.0,True,False,False,True,True,0.0,True,2848.8618,FTL,3 days 00:17:05,4.0,260225.0


In [10]:
# variables used for modeling
bool_column_names = [
 'FD_ENABLED',
 'EXCLUSIVE_USE_REQUESTED',
 'HAZARDOUS',
 'REEFER_ALLOWED',
 'STRAIGHT_TRUCK_ALLOWED',
 'LOAD_TO_RIDE_REQUESTED',
]

numerical_loggable_column_names = [
 'APPROXIMATE_DRIVING_ROUTE_MILEAGE',
 'PALLETIZED_LINEAR_FEET',
 'SECONDS_BETWEEN_ORDER_AND_DEADLINE',
 'LOAD_BAR_COUNT',
 'ESTIMATED_COST_AT_ORDER'
]

In [11]:
train=joinedDF.sample(frac=0.8,random_state=200)
test=joinedDF.drop(train.index)

In [12]:
xTrain = train[bool_column_names + numerical_loggable_column_names].to_numpy()
yTrain = train['NUMBER_OFFERS'].to_numpy()
xTest = test[bool_column_names + numerical_loggable_column_names].to_numpy()
yTest = test['NUMBER_OFFERS'].to_numpy()
xTrain

array([[True, True, False, ..., 605220.0, 0.0, 2830.0407],
       [True, True, False, ..., 82480.0, 0.0, 2866.6767],
       [True, True, False, ..., 636947.0, 0.0, 1685.7978],
       ...,
       [True, True, False, ..., 283893.0, 0.0, 1036.6698],
       [True, True, False, ..., 27154.0, 0.0, 4773.3612],
       [True, True, False, ..., 163136.0, 0.0, 2433.4173]], dtype=object)

In [13]:
reg = LinearRegression().fit(xTrain, yTrain)
preds = reg.predict(xTest).round()
mean_absolute_error(yTest,preds)

2.155251428212694

In [14]:
pd.Series(preds).value_counts()

3.0    11230
4.0     2328
2.0     2296
1.0       63
5.0       12
dtype: int64

In [15]:
avg_pred = [np.mean(nOffers_rec)] * len(yTest)
mean_absolute_error(yTest,avg_pred)

2.3729543716211947

In [16]:
test

,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,DELIVERY_TIME_CONSTRAINT,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE,diff_ORDER_PICKUP_DEADLINE,NUMBER_OFFERS,SECONDS_BETWEEN_ORDER_AND_DEADLINE
REFERENCE_NUMBER,,,,,,,,,,,,,,,,,,,
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,False,False,0.0,True,3024.8049,FTL,8 days 00:15:44,3.0,692144.0
"[\n ""0004b681b1c4f14dc289a36cf0590ffce5b6aa2220f6b340d550c952ea4caf23""\n]",2020-08-17 15:29:20,2020-08-18 15:30:00,NONE,900,276,2547.0,24.0,True,True,False,True,True,0.0,False,9245.7654,FTL,1 days 00:00:40,1.0,86440.0
"[\n ""000880c3b32626141c08b5040c79d5bdb72af4225f06a1f1615c686f97e9775f""\n]",2022-02-24 09:07:45,2022-02-28 14:00:00,NONE,716,515,611.0,52.0,True,True,False,False,False,0.0,False,2956.8993,FTL,4 days 04:52:15,1.0,363135.0
"[\n ""000d3a9bad1e2fa2c921b7565f21f5d31fe9291f02c9d5fb4495825fcfeb2dc3""\n]",2022-08-22 08:36:55,2022-09-07 13:00:00,SCHEDULE,305,704,503.0,52.0,True,False,False,True,True,0.0,False,1641.6411,FTL,16 days 04:23:05,0.0,1398185.0
"[\n ""0011ec19bd38340c2a86d26fee984485ed966a901ffc8bad78c00ba1a4a32fc8""\n]",2021-09-30 06:29:59,2021-10-05 07:00:00,SCHEDULE,296,281,139.0,46.0,True,False,False,True,True,0.0,False,896.8596,FTL,5 days 00:30:01,2.0,433801.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"[\n ""ffe54c86ce7ef7be88217ab660f339c45e46e1292dca424bb012130d100c30ea""\n]",2021-11-22 14:02:31,2021-11-29 13:00:00,SCHEDULE,303,282,256.0,100.0,True,True,False,True,True,0.0,True,1411.5309,FTL,6 days 22:57:29,0.0,601049.0
"[\n ""ffef1a34842ba031f5fbf07203651a9116a447edf08cc32453a36eac8d783296""\n]",2022-09-16 13:55:55,2022-09-20 15:00:00,SCHEDULE,841,604,1397.0,100.0,True,True,False,False,False,0.0,True,3099.6636,FTL,4 days 01:04:05,0.0,349445.0
"[\n ""fff53791fdea029938fbbc7f613aaf68505f9537129d2d67c5e48a20716a0d4a""\n]",2022-08-15 10:50:30,2022-08-16 12:00:00,SCHEDULE,303,296,166.0,48.0,True,True,False,False,False,0.0,True,965.5263,FTL,1 days 01:09:30,3.0,90570.0


In [17]:
len(preds)

15929

In [18]:
offers_orders = pd.merge(test, orders, on='REFERENCE_NUMBER', how='inner')
offers_orders['Estimated_Num_offers'] = preds.astype(int)
offers_orders = offers_orders.set_index('REFERENCE_NUMBER')

offers_orders

,ORDER_DATETIME_PST_x,PICKUP_DEADLINE_PST_x,DELIVERY_TIME_CONSTRAINT_x,ORIGIN_3DIGIT_ZIP_x,DESTINATION_3DIGIT_ZIP_x,APPROXIMATE_DRIVING_ROUTE_MILEAGE_x,PALLETIZED_LINEAR_FEET_x,FD_ENABLED_x,EXCLUSIVE_USE_REQUESTED_x,HAZARDOUS_x,...,EXCLUSIVE_USE_REQUESTED_y,HAZARDOUS_y,REEFER_ALLOWED_y,STRAIGHT_TRUCK_ALLOWED_y,LOAD_BAR_COUNT_y,LOAD_TO_RIDE_REQUESTED_y,ESTIMATED_COST_AT_ORDER_y,TRANSPORT_MODE_y,diff_ORDER_PICKUP_DEADLINE_y,Estimated_Num_offers
REFERENCE_NUMBER,,,,,,,,,,,,,,,,,,,,,
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,...,True,False,False,False,0.0,True,3024.8049,FTL,8 days 00:15:44,3
"[\n ""0004b681b1c4f14dc289a36cf0590ffce5b6aa2220f6b340d550c952ea4caf23""\n]",2020-08-17 15:29:20,2020-08-18 15:30:00,NONE,900,276,2547.0,24.0,True,True,False,...,True,False,True,True,0.0,False,9245.7654,FTL,1 days 00:00:40,3
"[\n ""000880c3b32626141c08b5040c79d5bdb72af4225f06a1f1615c686f97e9775f""\n]",2022-02-24 09:07:45,2022-02-28 14:00:00,NONE,716,515,611.0,52.0,True,True,False,...,True,False,False,False,0.0,False,2956.8993,FTL,4 days 04:52:15,3
"[\n ""000d3a9bad1e2fa2c921b7565f21f5d31fe9291f02c9d5fb4495825fcfeb2dc3""\n]",2022-08-22 08:36:55,2022-09-07 13:00:00,SCHEDULE,305,704,503.0,52.0,True,False,False,...,False,False,True,True,0.0,False,1641.6411,FTL,16 days 04:23:05,3
"[\n ""0011ec19bd38340c2a86d26fee984485ed966a901ffc8bad78c00ba1a4a32fc8""\n]",2021-09-30 06:29:59,2021-10-05 07:00:00,SCHEDULE,296,281,139.0,46.0,True,False,False,...,False,False,True,True,0.0,False,896.8596,FTL,5 days 00:30:01,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"[\n ""ffe54c86ce7ef7be88217ab660f339c45e46e1292dca424bb012130d100c30ea""\n]",2021-11-22 14:02:31,2021-11-29 13:00:00,SCHEDULE,303,282,256.0,100.0,True,True,False,...,True,False,True,True,0.0,True,1411.5309,FTL,6 days 22:57:29,2
"[\n ""ffef1a34842ba031f5fbf07203651a9116a447edf08cc32453a36eac8d783296""\n]",2022-09-16 13:55:55,2022-09-20 15:00:00,SCHEDULE,841,604,1397.0,100.0,True,True,False,...,True,False,False,False,0.0,True,3099.6636,FTL,4 days 01:04:05,3
"[\n ""fff53791fdea029938fbbc7f613aaf68505f9537129d2d67c5e48a20716a0d4a""\n]",2022-08-15 10:50:30,2022-08-16 12:00:00,SCHEDULE,303,296,166.0,48.0,True,True,False,...,True,False,False,False,0.0,True,965.5263,FTL,1 days 01:09:30,3


In [19]:
joinedTable = test.join(offers.set_index('REFERENCE_NUMBER').drop(columns = ['TRANSPORT_MODE']), how = 'inner')
joinedTable

,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,DELIVERY_TIME_CONSTRAINT,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,...,OFFER_TYPE,SELF_SERVE,IS_OFFER_APPROVED,AUTOMATICALLY_APPROVED,MANUALLY_APPROVED,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,VALID
REFERENCE_NUMBER,,,,,,,,,,,,,,,,,,,,,
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,...,quote,False,True,False,True,False,True,True,False,True
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,...,quote,False,False,False,False,False,False,False,False,False
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,...,quote,False,False,False,False,False,False,False,False,True
"[\n ""0004b681b1c4f14dc289a36cf0590ffce5b6aa2220f6b340d550c952ea4caf23""\n]",2020-08-17 15:29:20,2020-08-18 15:30:00,NONE,900,276,2547.0,24.0,True,True,False,...,quote,False,False,False,False,False,True,True,False,True
"[\n ""000880c3b32626141c08b5040c79d5bdb72af4225f06a1f1615c686f97e9775f""\n]",2022-02-24 09:07:45,2022-02-28 14:00:00,NONE,716,515,611.0,52.0,True,True,False,...,quote,False,True,True,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"[\n ""fff6f947ed25527c7ac3e4c2e598bbfaa17fa4f60e8bea8a64c69b216424da53""\n]",2022-08-22 07:52:21,2022-08-30 11:00:00,SCHEDULE,305,720,606.0,53.0,True,False,False,...,quote,True,False,False,False,False,False,False,True,True
"[\n ""fffc15629ba0aa26f5252d1b619e01cf10306f97b440027ee7dc899a4f59b107""\n]",2021-03-29 15:39:19,2021-03-31 11:00:00,SCHEDULE,303,945,2471.0,47.0,True,True,False,...,quote,False,False,False,False,False,False,False,False,True
"[\n ""fffc15629ba0aa26f5252d1b619e01cf10306f97b440027ee7dc899a4f59b107""\n]",2021-03-29 15:39:19,2021-03-31 11:00:00,SCHEDULE,303,945,2471.0,47.0,True,True,False,...,quote,False,False,False,False,False,False,False,False,True


In [20]:
nonidx_joinedDf = joinedTable.reset_index()
nonidx_joinedDf['ESTIMATED_COST_AT_ORDER']

uniqueRefs = nonidx_joinedDf['REFERENCE_NUMBER'].unique()
uniqueRefs

array(['[\n  "0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371"\n]',
       '[\n  "0004b681b1c4f14dc289a36cf0590ffce5b6aa2220f6b340d550c952ea4caf23"\n]',
       '[\n  "000880c3b32626141c08b5040c79d5bdb72af4225f06a1f1615c686f97e9775f"\n]',
       ...,
       '[\n  "fff53791fdea029938fbbc7f613aaf68505f9537129d2d67c5e48a20716a0d4a"\n]',
       '[\n  "fff6f947ed25527c7ac3e4c2e598bbfaa17fa4f60e8bea8a64c69b216424da53"\n]',
       '[\n  "fffc15629ba0aa26f5252d1b619e01cf10306f97b440027ee7dc899a4f59b107"\n]'],
      dtype=object)

In [21]:
train, test = train_test_split(nonidx_joinedDf, test_size=0.2)
y = np.array(train['RATE_USD'])
X = np.array(train[['SECONDS_BETWEEN_ORDER_AND_DEADLINE','ESTIMATED_COST_AT_ORDER']])
reg = LinearRegression().fit(X, y)
predictions = reg.predict(nonidx_joinedDf[['SECONDS_BETWEEN_ORDER_AND_DEADLINE','ESTIMATED_COST_AT_ORDER']])
predictions

/Users/nimayazdani/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([3627.34385549, 3627.34385549, 3627.34385549, ..., 7906.97911173,
       7906.97911173, 7906.97911173])

In [22]:
nonidx_joinedDf['My_ESTIMATED_COST_AT_ORDER'] = predictions
nonidx_joinedDf

,REFERENCE_NUMBER,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,DELIVERY_TIME_CONSTRAINT,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,...,SELF_SERVE,IS_OFFER_APPROVED,AUTOMATICALLY_APPROVED,MANUALLY_APPROVED,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,VALID,My_ESTIMATED_COST_AT_ORDER
0,"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176...",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,...,False,True,False,True,False,True,True,False,True,3627.343855
1,"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176...",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,...,False,False,False,False,False,False,False,False,False,3627.343855
2,"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176...",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,...,False,False,False,False,False,False,False,False,True,3627.343855
3,"[\n ""0004b681b1c4f14dc289a36cf0590ffce5b6aa22...",2020-08-17 15:29:20,2020-08-18 15:30:00,NONE,900,276,2547.0,24.0,True,True,...,False,False,False,False,False,True,True,False,True,10409.877294
4,"[\n ""000880c3b32626141c08b5040c79d5bdb72af422...",2022-02-24 09:07:45,2022-02-28 14:00:00,NONE,716,515,611.0,52.0,True,True,...,False,True,True,False,False,True,True,False,True,3585.489653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53147,"[\n ""fff6f947ed25527c7ac3e4c2e598bbfaa17fa4f6...",2022-08-22 07:52:21,2022-08-30 11:00:00,SCHEDULE,305,720,606.0,53.0,True,False,...,True,False,False,False,False,False,False,True,True,2334.060815
53148,"[\n ""fffc15629ba0aa26f5252d1b619e01cf10306f97...",2021-03-29 15:39:19,2021-03-31 11:00:00,SCHEDULE,303,945,2471.0,47.0,True,True,...,False,False,False,False,False,False,False,False,True,7906.979112
53149,"[\n ""fffc15629ba0aa26f5252d1b619e01cf10306f97...",2021-03-29 15:39:19,2021-03-31 11:00:00,SCHEDULE,303,945,2471.0,47.0,True,True,...,False,False,False,False,False,False,False,False,True,7906.979112
53150,"[\n ""fffc15629ba0aa26f5252d1b619e01cf10306f97...",2021-03-29 15:39:19,2021-03-31 11:00:00,SCHEDULE,303,945,2471.0,47.0,True,True,...,False,False,False,False,False,False,False,False,True,7906.979112


In [23]:
len(uniqueRefs)

13562

## Conduct Sec Method with a prior using model that estimates the # of offers and a model that estimates cost

In [25]:
rates = []
costs = []

for i in uniqueRefs:
    Estimated_Num_offers = offers_orders.loc[i]['Estimated_Num_offers']
    SecNum =  math.ceil(Estimated_Num_offers / np.e)
    AllOffers = nonidx_joinedDf[nonidx_joinedDf['REFERENCE_NUMBER'] == i].sort_values(by='CREATED_ON_HQ', ascending=True)
    a =0 
    if Estimated_Num_offers == 1 and len(AllOffers) > 0:
        record = AllOffers.iloc[0]['RATE_USD']
        rates.append(record)
        break
        
    elif (Estimated_Num_offers > 1) and (len(AllOffers) > 0):
        for n in range(SecNum-1):
            est_cost = AllOffers.iloc[n]['My_ESTIMATED_COST_AT_ORDER']
            actual_cost = AllOffers.iloc[n]['RATE_USD']
            if actual_cost < est_cost:
                record = actual_cost
                rates.append(record)
                a=1
                break
        if a == 0 :
            record = min(AllOffers.iloc[:SecNum]['RATE_USD'])
            for num in AllOffers['RATE_USD']:
                if num < record or (num == np.array(AllOffers['RATE_USD'])[-1]):
                    record = num
                    rates.append(record)
                    break

In [26]:
our_avg_rate = np.mean(rates)
our_avg_rate

3539.216146712803

In [27]:
Flock_avg_rate = np.mean(nonidx_joinedDf[nonidx_joinedDf['LOAD_DELIVERED_FROM_OFFER'] == True]['RATE_USD'])
Flock_avg_rate

3377.345922545625